---

DATE:   January 11, 2022

AUTHOR: Benedikt WELLES

FILE:   useWiresharkParser.ipynb

---

# General Information

This Jupyter-Notebook is used to supplement development and bug-fixing of the Python-Script \<ptp_sim_aut_ver_tool.py\>

This file is intended to be used as both a means to troubleshoot and to document bugs found within the script used to automatically
analyse captured PTP network traffic.

--- 

## Important Note: 

+ Use the \<Restart\> button at the top of the VS Code window ***after*** changing something in the script \<ptp_sim_aut_ver_tool.py\>
+ Afterwards run the code cell below this textbox to re-import the (changed) main script.

In [ ]:
from ptp_sim_aut_ver_tool import parseFile

---

---

# Documentation

The following section of this Jupyter-Notebook is meant to provide information about the iterative process of implementing new 
functionality for the \<ptp_sim_aut_ver_tool.py\> script, as well as to document the bug-fixing of existing functionality.

### Structure

The following section consists of Markdown and Python-code cells:

+ Text written in the Markdown cells is meant to introduce an identified problem, give information about found solutions as well as a 
closing remark explaining how the encountered problem was fixed. 

+ The cells of Python-code accompanying Markdown sections are intended to show that the specific problem mentioned within the text has 
been solved.

---

---

# Issues with misinterpreted IPv4 Addresses

Input File: "testdata/optimusPrime_TC_R0017.pcapng.log"

Regarding the apparent misinterpretation of the <ip.src> value of ICMP Packets

## Problem

Certain packets found within the input file specified above were interpreted to have problematic IPv4 <ip.src> values. 

For example: Packet 42227 is an ICMP packet, which is displayed with the wireshark error-message "destination unreachable (port unreachable)"
+ Its source is 10.0.0.203, its destination 10.0.0.204 ... which would correlate with problematic <ip.src=='10.0.0.203,10.0.0.204'>
+ Such ICMP packets are displayed even if the tshark filter <-Y "ptp"> is used (this seems to be intended behaviour?)

One can see that <10.0.0.203,10.0.0.204> ***is not*** a valid IPv4 address

## Solution

Two different ways of circumventing this particular probelm have been found: 

When invoking tshark for the very first time (to determine unique ip.src values) one can utilize ...
+ the display filter <-Y "ptp and not icmp">
    + now ICMP packets are delibaretely ignored
+ the additional option <-E occurrence=f>
    + the "," within the problematic IPv4 addresses is interpreted as a seperator ... Therefore only the <ip.src> part of the overall address is used

## Closing Remarks

This solution merely circumvents the problems mentioned above. An actual answer as to ***why*** tshark would wrongly interpret the field ip.src for a ICMP packets with the error code "destination unreachable (port unreachable)" ***was not found***

In [ ]:
inputFileName = "./testdata/optimusPrime_TC_R0017.pcapng.log"

parseFile(inputFileName)

---
# Issues differentiating IPv4 and IPv6 Addresses

Input File: "testdata/walle_TC_R0028.pcapng"

## Problem

A previous version of the underlying script was not capable of differentiating between IPv4 and IPv6 addresses, let alone parse IPv6 addresses. 

## Solution

The underlying script was updated to differentiate between IPv4 and IPv6 addresses and is now able to parse both types of addresses. 

An extra invocation of tshark is used to determine the value of the <eth.type> field of the first PTP packet found within a specified input file. Afterwards, boolean 
flags are set according to the IPvX address type found within this first PTP packet. Accroding to these boolean flags subsequent tshark invocations acquire relevant information
from the appropriate fields of a PTP message. 

These relevant fields for the distinction between IPv4 and IPv6 packtes are as follows:
+ IPv4 ... ip.src
+ IPv6 ... ipv6.src

Other than that, no functionality of the underlying script has been changed. 

## TODO

TODO: implement a future check whether ***all*** PTP packtes found in an input file utilize the same type of IPvX addresses.

## Closing Remarks

TODO: update text

Updating the underlying script to differentiate between IPv4 and IPv6 addresses was easier than expected. 

In [ ]:
inputFileName = "./testdata/walle_TC_R0028.pcapng"

parseFile(inputFileName)

---

# Issues with not (yet) supported types of PTP Messages

Input file: optimusPrime_TC_R0044.pcapng.log

## Problem

A previous version of the underlying script was not able to run properly if PTP messages of not (yet) supported types were found.

In such a case the iterating loops handling the generation of data frames according to identified PTP message types were not executed as intended. 
For this reason data frames that should have been created and filled with information corresponding to the supported PTP message types were not
created/filled as intended and the underlying script crashed due to an access of a data frame column which did not exist.

## Solution

The iterating loops handling the generation of data frames were reworked to properly accomodate not (yet) supported types of PTP messages ***without*** 
impeding the creation of other data frames

## Closing Remarks

The purpose of this update of the underlying script was to simply fix the problem regarding the iterating loops responsible for generating and filling
data frames according to the identified PTP message types. 

However, the script ***was not yet*** updated to sensibly support all types of PTP messages. 

In [ ]:
inputFileName = "./testdata/optimusPrime_TC_R0044.pcapng.log"

parseFile(inputFileName)

---

# Issues with Input Files containing no PTP Messages

Input file: c3po_TC_R0033.pcapng.log

## Problem

There are no PTP messages within this file, which lead to a crash of a previous version of the underlying script.

## Solution

A boolean flag was added to represent whether or not eligible PTP messages were found within a given input file. If no eligible PTP messages
are found the script displays an informative message and exits. This check was implemented at the beginning of the script:

As per a previous update to the underlying script another data frame was added to allow for a check of the value of the field <eth.type>. If the data
frame intentionally added for this purpose ends up being empty, no eligible PTP messages were found within the given input file. 

## Closing Remarks

As of now this check only happens once at the beginning of the script. If the data frame through which <eth.type> shall be determined ends up being ***not empty*** it is assumed no PTP messages were found within the given input file. 

In [ ]:
inputFileName = "./testdata/c3po_TC_R0033.pcapng.log"

parseFile(inputFileName)